## Sensitive analysis:


In [2]:
from sast.sast import *
from sktime.datasets import load_UCR_UEA_dataset, tsc_dataset_names
from sktime.classification.kernel_based import RocketClassifier
import time
import pandas as pd


/home/nirojasvar/anaconda3/envs/rsast_env/lib/python3.10/site-packages/mass_ts/_mass_ts.py:17: UserWarning: GPU support will not work. You must pip install mass-ts[gpu].
  warnings.warn(


### Select Datasets for hypertunning RSAST

It is runned RSAST in a set of UCR datasets with a predefined number of runs ("runs"). Then, it is selected a range ("range_total") between 1, 10, 100 and 10000 for the selected dataset.

In [3]:
df_result=pd.read_csv("df_overall_tunning.csv")

In [4]:
# create and export a pivot table with the mean of score
pivot_table = pd.pivot_table(df_result, values='accuracy', index=['classifier_name','dataset_name'], aggfunc='mean')
pivot_table.to_csv("df_pivot_tunning_hyper.csv") 


In [5]:
pd.pivot_table(df_result, values='accuracy', index=['len_method','rpoint'],columns=['nb_per_class'], aggfunc='mean')

nb_per_class            1         10        30        50        100
len_method rpoint                                                  
ACF        1       0.497841  0.708890  0.708885  0.708151  0.682829
           10      0.616813  0.714102  0.715459  0.727506  0.740720
           30      0.666515  0.695251  0.732220  0.732438  0.737218
           50      0.678559  0.692262  0.735954  0.737153  0.741041
           100     0.649737  0.686801  0.738057  0.744294  0.739712
ACF&PACF   1       0.792302  0.937510  0.936419  0.923446  0.947753
           10      0.913537  0.951518  0.955547  0.958620  0.955199
           30      0.903977  0.935287  0.956310  0.957993  0.961674
           50      0.907097  0.941089  0.961310  0.965962  0.968601
           100     0.923780  0.934018  0.958705  0.964521  0.962629
Max ACF    1       0.506316  0.705864  0.708595  0.700968  0.708032
           10      0.618590  0.705332  0.718137  0.722223  0.731809
           30      0.671436  0.709344  0.729562  0.741165  0.746102
           50      0.651240  0.702905  0.732182  0.738218  0.742546
           100     0.641867  0.689811  0.745962  0.744135  0.743264
Max PACF   1       0.708984  0.852323  0.890562  0.884415  0.908131
           10      0.882327  0.916729  0.939742  0.940940  0.948652
           30      0.889341  0.921635  0.942760  0.939314  0.941223
           50      0.905339  0.913444  0.943151  0.950344  0.948079
           100     0.895569  0.913374  0.940295  0.942227  0.949000
None       1       0.708062  0.880030  0.891021  0.891002  0.896723
           10      0.829351  0.912370  0.922951  0.925751  0.931124
           30      0.823373  0.895534  0.918721  0.930958  0.930956
           50      0.760984  0.902098  0.921956  0.923308  0.944181
           100     0.838437  0.854120  0.911089  0.932020  0.931263
PACF       1       0.855042  0.936335  0.938138  0.923811  0.928196
           10      0.874332  0.927908  0.958202  0.954423  0.960651
           30      0.899152  0.955459  0.955841  0.962855  0.962490
           50      0.908659  0.946535  0.956188  0.957490  0.964660
           100     0.898729  0.942648  0.956205  0.960945  0.964538

In [6]:
# create a pivot table with the mean of score by hyperparameter
pd.pivot_table(df_result, values='accuracy', index=['rpoint'],columns=['nb_per_class'], aggfunc='mean')

nb_per_class,1,10,30,50,100
rpoint,,,,,
1,0.678091,0.836825,0.845603,0.838632,0.845277
10,0.789158,0.854660,0.868340,0.871577,0.878026
30,0.808966,0.852085,0.872569,0.877454,0.879944
50,0.801980,0.849722,0.875123,0.878746,0.884851
100,0.808020,0.836795,0.875052,0.881357,0.881734


In [ ]:
# filter by method with ACF&PACF
# create a pivot table with the mean of score by hyperparameter
df_result_acc=df_result[df_result["len_method"]=="ACF&PACF"]
pd.pivot_table(df_result_acc, values='accuracy', index=['rpoint'],columns=['nb_per_class'], aggfunc='mean')

In [ ]:
# create a pivot table with the variatioon of score by hyperparameter
pd.pivot_table(df_result_acc, values='accuracy', index=['rpoint'],columns=['nb_per_class'], aggfunc='var')

#### Analysis of time complexity

##### Overall time complexity

In [ ]:
# create a pivot table with the mean of time spent in minutes
pd.pivot_table(df_result_acc, values='time', index=['rpoint'],columns=['nb_per_class'], aggfunc='mean')

##### Calculate weights time complexity

In [ ]:
pd.pivot_table(df_result_acc, values='cweights_time', index=['rpoint'],columns=['nb_per_class'], aggfunc='mean')

##### Finding subsequences time complexity

In [ ]:
pd.pivot_table(df_result_acc, values='fsubsequence_time', index=['rpoint'],columns=['nb_per_class'], aggfunc='mean')

##### Transform Dataset time complexity

In [ ]:
pd.pivot_table(df_result_acc, values='tdataset_time', index=['rpoint'],columns=['nb_per_class'], aggfunc='mean')

##### Classifier time complexity

In [ ]:
pd.pivot_table(df_result_acc, values='tclassifier_time', index=['rpoint'],columns=['nb_per_class'], aggfunc='mean')